<a href="https://colab.research.google.com/github/vadim-privalov/Neiroset_Novosibirsk/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%22%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%82%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0_%22_%D0%BD%D0%B0_Kaggle%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ООО "Университет Цифровых технологий платформа 3"

## Домашнее задание к уроку "Нейронные сети для работы с текстами. Часть 1."

Определение тональности текста.
Цель состоит в том, чтобы определить тональность текста на основе предоставленных данных. Данные представляют набор текстовых данных отзывов на покупку мобильных телефонов и аксессуаров. Где 1 — положительный отзыв, 0 — отрицательный отзыв.

Загрузите из датасета текстовую базу отзывов на покупку мобильных телефонов и аксессуаров, а также базу для тестирования. Подайте дата сет в нейронную сеть, постаравшись добиться максимальной точности распознавания. Для улучшения обучения можно использовать также дополнительные методы, не рассмотренные в занятиях, слои, функции активации, менять параметры оптимизатора Adam и прочие гипер параметры. Результаты рекомендуется заносить в таблицу для себя, чтобы можно было потом воспроизвести лучший вариант. Предсказание Вашей сети необходимо отправить в виде специального файла с расширением csv, чтобы войти в рейтинговую таблицу соревнования. Ответ можно улучшить и подавать результат несколько раз.

Содержание датасета: Набор текстовых данных содержат отзывы покупателей на покупку мобильных телефонов и аксессуаров и анализ тональности по 5-ти бальной шкале. После загрузки необходимо преобразовать шкалу тональности в двумерную, позитивная и негативная, с кодированием где 1 — положительный отзыв, 0 — отрицательный отзыв.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pprint import pprint
from google.colab import files

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf # импортируем tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer,tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Embedding, SpatialDropout1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
import zipfile # для работы с архивами zip

import os
#Отрисовывать изображения в ноутбуке, а не в консоль или файл
%matplotlib inline
# устанавливаем свежий Kaggle API
!pip install --upgrade --force-reinstall --no-deps kaggle


     |████████████████████████████████| 58 kB 5.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=3b66c57b73da2d55b82aeecfbfbb5a6a9ca6bdf2ecd52feadfb15c264712d475
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Анализ тональности текста


### Ссылка на сревнование: 
https://www.kaggle.com/t/82c8fec502b3466bbf725731055646c2




###Устанавливаем ключ Каггл, чтобы участвовать в соревнованиях. Это нужно сделать один раз.
Нужно зарегистрироваться на Каггл (https://www.kaggle.com/) и скачать ключ для взаимодействия с API соревновательной системы Каггла. После регистрации, справа появятся Ваши имя и фамилия - ссылка на персональную страницу. Нажмите на них. Когда страница загрузится, справа увидите Account. Нажимаете на надпись, затем прокручиваете страницу вниз, до вкладки API, там нажимаете Create New API Token. Ключ - это файл kaggle.json, который скачается после нажатия. Нужно скопировать его в папку Google Drive, предварительно её создав: My Drive/datasets/kaggle/ (из Колаба её адрес, который будем указывать для загрузки, такой: /content/drive/My Drive/datasets/kaggle/).

In [ ]:
def Init_Kaggle():
    try:
        # Создаём правильную папку для ключа Каггл
        # в Колабе, она должна быть такой по требованию Каггл
        !mkdir ~/.kaggle
        # переходим в папку, теперь это путь по умолчанию
        %cd ~/.kaggle
        # запускаем - подключаем Google Drive - будем скачивать с него, так намного быстрее и удобнее
        # после запуска переходим по ссылке, которая появится, для идентификации
        # копируем оттуда код authorization code и вставляем здесь в окошко
        drive.mount('/content/drive')
        # нужно зарегистрироваться на Каггл и скачать ключ для взаимодействия с API
        # соревновательной системы Каггла kaggle.json. Нужно скопировать его в папку
        # Google Drive, предварительно её создав: /content/drive/My Drive/datasets/kaggle/
        !cp '/content/drive/My Drive/datasets/kaggle/kaggle.json' '/root/.kaggle/kaggle.json'
        # выводим версию API Kaggle:
        !kaggle --version
        !ls
        print(' Ключи Kaggle успешно установлены! ')
    except:
        print(' При установке ключей Kaggle произошла ошибка! ') 

In [ ]:
def Load_data_from_Kaggle():
    # считаем заполненную базу данных из kaggle
    %cd ~/.kaggle
    print('скачиваем данные с Каггл:')
    !kaggle competitions download -c udt-3-text-work
    print('загружаем из архива, распаковываем:')
    z = zipfile.ZipFile('udt-3-text-work.zip', 'r')
    z.extractall()
    # удаляем архив, чтобы не занимал место на диске
    !rm udt-3-text-work.zip
    #Считываем данные с помощью pandas из двух файлов
    reviews = pd.read_csv('reviews_mobile_phone_train.csv')
    test_data = pd.read_csv('reviews_mobile_phone_test.csv')
    
    return reviews, test_data

In [ ]:
Init_Kaggle() # устанавливаем кючи кагл

/root/.kaggle
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Kaggle API 1.5.12
kaggle.json
 Ключи Kaggle успешно установлены! 


In [ ]:
reviews, test_data = Load_data_from_Kaggle() # считаем заполненную базу данных из kaggle
reviews.head()

/root/.kaggle
скачиваем данные с Каггл:
 80% 17.0M/21.2M [00:00<00:00, 27.3MB/s]
100% 21.2M/21.2M [00:00<00:00, 27.3MB/s]
загружаем из архива, распаковываем:


,text,label
0,Хорошее соотношение Цена-Качество! Минимум пон...,5
1,"Крупный экран, кнопки клавиатуры и цифры на эк...",2
2,"Пользуюсь Sharp Aquos S2. Достойный агрегат, м...",5
3,-Большой и яркий экран-Долго держит заряд-Стил...,5
4,"Тонкий корпус, звук с динамиков и в наушниках,...",4


In [ ]:
# выведем информацию о колонках
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53265 entries, 0 to 53264
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    53265 non-null  object
 1   label   53265 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 832.4+ KB


In [ ]:
# посмотрим, сколько у нас отзывов с какими оценками
reviews['label'].value_counts()

5    24823
4    12043
3     6606
1     5030
2     4763
Name: label, dtype: int64

In [ ]:
def Data_Update(reviews): #преодбработка данных
    global NUM_WORDS
    global tokenizer
    # создадим словарь для преобразования оценки из шкалы  5-1 в  шкалу positive - negative
    # используем 0 для класса негативных отзывов, 1 для класса позитивных отзывов
    label_to_binary = {5: 1, 
                    4: 1, 
                    3: 0, 
                    2: 0, 
                    1: 0}
    # создадим новую колонку в DataFramкe
    reviews['is_positive'] = reviews['label'].apply(lambda x: label_to_binary[x])
    # выделим наши тексты (признаковые описания) и класс, который нужно предсказывать
    X = reviews['text']
    Y = reviews['is_positive']
    # разделим наши данные на обучающую и валидационную выборки
    train_index, val_index = train_test_split(np.arange(reviews.shape[0]))
    x_train_raw, x_val_raw = X[train_index], X[val_index]
    y_train, y_val = Y[train_index], Y[val_index] 
    NUM_WORDS = 5000 # константа, максимальное количество слов, которые будет учитывать наша модель

    # создадим наш Токенайзер
    tokenizer = Tokenizer(num_words=NUM_WORDS, # максимальное количество слов, которые будет учитывать наш токенайзер
                        filters='"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', # символы, которые он токенайзер будет фильтровать
                        lower=True,  #приводятся ли все символы к нижнему регситру
                        split=' ',  # символ, по которому происходит разделение на слова (токены)
                        char_level=False,  # являются ли токенами отдельные буквы
                        oov_token='UNKNOWN' # токен для неизвестных слов 
                        )
    tokenizer.fit_on_texts(x_train_raw) # обучим наш токенайзер на обучающих текстах
    # Преобразуем наши текстовые данные в формат One Hot Encoding
    x_train_01 = tokenizer.texts_to_matrix(x_train_raw)
    x_val_01 = tokenizer.texts_to_matrix(x_val_raw)
    return x_train_01, y_train, x_val_01, y_val


In [ ]:
x_train_01, y_train, x_val_01, y_val = Data_Update(reviews)

In [ ]:
x_train_01.shape

(39948, 5000)

# Создание и обучение модели

In [ ]:
# создадим простую модель с полносвязными слоем
def Dense_model():
    model01 = Sequential()
    model01.add(Dense(6000, input_dim=NUM_WORDS, activation='relu'))
    model01.add(BatchNormalization())
    model01.add(Dropout(0.4))
    model01.add(Dense(3000 ,activation='tanh'))
    model01.add(Dense(1000 ,activation='tanh'))
    model01.add(Dropout(0.3))
    model01.add(Dense(100 ,activation='relu'))
    model01.add(Dense(1 ,activation='sigmoid'))
    model01.compile(optimizer=Adam(learning_rate=0.001),  loss='binary_crossentropy',  metrics=['accuracy'])

    return model01

In [ ]:
# обучим нашу модель
model_text_tonality = Dense_model()
model_text_tonality.fit(x = x_train_01,  y = y_train, epochs = 15, verbose = 1, validation_data= (x_val_01, y_val))

Epoch 1/15
1249/1249 [==============================] - 33s 26ms/step - loss: 0.4561 - accuracy: 0.8064 - val_loss: 0.3480 - val_accuracy: 0.8518
Epoch 2/15
1249/1249 [==============================] - 31s 25ms/step - loss: 0.2914 - accuracy: 0.8789 - val_loss: 0.3445 - val_accuracy: 0.8642
Epoch 3/15
1249/1249 [==============================] - 32s 25ms/step - loss: 0.2078 - accuracy: 0.9167 - val_loss: 0.3646 - val_accuracy: 0.8615
Epoch 4/15
1249/1249 [==============================] - 32s 25ms/step - loss: 0.1313 - accuracy: 0.9516 - val_loss: 0.4072 - val_accuracy: 0.8566
Epoch 5/15
1249/1249 [==============================] - 32s 25ms/step - loss: 0.0883 - accuracy: 0.9681 - val_loss: 0.4924 - val_accuracy: 0.8581
Epoch 6/15
1249/1249 [==============================] - 32s 26ms/step - loss: 0.0727 - accuracy: 0.9744 - val_loss: 0.5040 - val_accuracy: 0.8566
Epoch 7/15
1249/1249 [==============================] - 32s 26ms/step - loss: 0.0574 - accuracy: 0.9800 - val_loss: 0.5501 -

In [ ]:
# сохраним модель 
model_text_tonality.save('/content/drive/MyDrive/datasets/model_text_tonality_03.h5') 

##Создадим предсказание обученной модели.

In [ ]:
def Prediction_model(test_data): # предсказываем значения
    try:
        # x_test_raw = pd.Series(test_data_reviews)
        x_test_raw = test_data['text']
        x_test_tok = tokenizer.texts_to_matrix(x_test_raw)
        
        # получим предсказания модели
        pred = model_text_tonality.predict(x_test_tok)
        pred = np.round(pred).astype(np.int)
        print('Прогноз тестовых данных сформирован успешно!')
        return pred 
    except:
        print('При формировании прогноза на тестовых данных произошла ошибка!')
        return 0

In [ ]:
# оформить результат и отправить на Kaggle
def Send_results(pred_data_my):
    try:
        predUnscaled = pred_data_my.squeeze()
        # создаём датафрейм в нужном формате
        submission = pd.DataFrame({"Id":range(1,len(predUnscaled)+1),"Label":predUnscaled})
        
        submission.head()
        # сохраняем его как csv
        submission = submission.to_csv('./Submission_review_phone.csv', sep=',', index=False, header=True)
        # отправляем на соревнование
        !kaggle competitions submit -c udt-3-text-work -f Submission_review_phone.csv -m "Message"
        print('\n Расчитанные данные успешно отправлены! ')
        
    except:
        print('Отправка расчетных данных не прошла! ')  

In [ ]:
prediction_data = Prediction_model(test_data) # получим прогноз на тестовой выбоке

Прогноз тестовых данных сформирован успешно!


In [ ]:
Send_results(prediction_data) # оформить результат и отправить на Kaggle

100% 5.76k/5.76k [00:04<00:00, 1.33kB/s]
Successfully submitted to Университет Цифровых Технологий платформа 3
 Расчитанные данные успешно отправлены! 
